In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon

In [2]:
pwd

u'/Users/macbook/Desktop/school/spring/bigdata/spark-2.1.0-bin-hadoop2.6/Analyzing_Noise_in_NYC/py'

In [3]:
ct = gpd.read_file('../data/nyct2010_17a/nyct2010.shp')

In [4]:
ct.head(2)

,BoroCT2010,BoroCode,BoroName,CDEligibil,CT2010,CTLabel,NTACode,NTAName,PUMA,Shape_Area,Shape_Leng,geometry
0,5000900,5,Staten Island,I,000900,9,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016794,"POLYGON ((962269.1260375977 173705.5001831055,..."
1,1009800,1,Manhattan,I,009800,98,MN19,Turtle Bay-East Midtown,3808,1.906016e+06,5534.199811,"POLYGON ((994133.507019043 214848.8975830078, ..."


In [5]:
noise = pd.read_csv('/Users/kristikorsberg/Downloads/311_Service_Requests_from_2010_to_Present.csv')

IOError: File /Users/kristikorsberg/Downloads/311_Service_Requests_from_2010_to_Present.csv does not exist

In [ ]:
noise.head(2)

In [ ]:
noise.columns

In [ ]:
noise.head(2)

In [ ]:
noise = noise[['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Complaint Type', 'Descriptor', 'Incident Zip', \
               'X Coordinate (State Plane)', 'Y Coordinate (State Plane)']]

In [ ]:
noise = noise.dropna()

In [ ]:
geometry = [Point(xy) for xy in zip(noise['X Coordinate (State Plane)'], noise['Y Coordinate (State Plane)'])]
noise = noise.drop(['X Coordinate (State Plane)', 'Y Coordinate (State Plane)'], axis=1)
crs = {'init': 'epsg:2263'}
geo_noise = gpd.GeoDataFrame(noise, crs=crs, geometry=geometry)

In [ ]:
print geo_noise.crs
print ct.crs

In [ ]:
ct = ct.to_crs({'init':'epsg:2263'})

In [ ]:
print geo_noise.crs
print ct.crs

In [ ]:
ct = ct[['geometry', 'CT2010', 'Shape_Area', 'Shape_Leng']]

In [ ]:
ct.columns

In [ ]:
noise_simple = geo_noise[['Unique Key', 'geometry']]

In [ ]:
#http://geoffboeing.com/2016/10/r-tree-spatial-index-python/
precise_matches = {}
spatial_index = geo_noise.sindex
for i, CT2010 in enumerate(ct.CT2010):
    possible_matches_index = list(spatial_index.intersection(ct['geometry'][i].bounds))
    possible_matches = geo_noise.iloc[possible_matches_index]
    precise_matches[CT2010] = possible_matches[possible_matches.intersects(ct['geometry'][i])]
    #precise_matches[CT2010]['CT2010'] = CT2010 

In [ ]:
noise_ct = pd.concat(precise_matches).reset_index()

In [ ]:
noise_ct.head(2)

In [ ]:
noise_ct = noise_ct.rename(columns={'level_0':'CT2010'})
noise_ct = noise_ct.drop('level_1',axis=1)

In [ ]:
noise_ct.head(2)

In [ ]:
noise_ct['CT2010'] = noise_ct['CT2010'].astype(str)

In [ ]:
noise_ct['Census Tract'] = noise_ct['CT2010'].apply(lambda s: s.lstrip("0"))

In [ ]:
noise_ct.head(2)

In [ ]:
pop = pd.read_csv('../data/New_York_City_Population_By_Census_Tracts.csv')

In [ ]:
pop.head(2)

In [ ]:
pop = pop[pop['Year']==2010]

In [ ]:
pop['Census Tract'] = pop['Census Tract'].astype(str)
noise_ct['Census Tract'] = noise_ct['Census Tract'].astype(str)

In [ ]:
pop.head(2)

In [ ]:
ct_noise_pop = pd.merge(noise_ct, pop, on='Census Tract', how='outer')

In [ ]:
ct_noise_pop.head(2)

In [ ]:
type(ct_noise_pop)

In [ ]:
print len(ct_noise_pop['Unique Key'].unique())
print len(noise['Unique Key'].unique())
print len(noise_ct['Unique Key'].unique())

In [ ]:
bbl = pd.read_csv('../data/ct_bbl_avg.csv')
bbl = bbl.drop('Unnamed: 0',axis=1)
bbl.head()

In [ ]:
ct_noise_pop.head(2)

In [ ]:
ct_noise_pop.CT2010 = ct_noise_pop.CT2010.astype(str)
dropped = []
for elem in ct_noise_pop.CT2010:
    dropped.append(elem.lstrip('00'))

In [ ]:
dropped[:10]

In [ ]:
ct_noise_pop.CT2010 = dropped

In [ ]:
ct_noise_pop.head(2)

In [ ]:
bbl.CT2010 = bbl.CT2010.astype(str)
merged = pd.merge(bbl, ct_noise_pop, on='CT2010')

In [ ]:
print len(merged)
print len(bbl)
print len(ct_noise_pop)

In [ ]:
merged.head()

In [ ]:
merged.columns

In [ ]:
merged = merged.drop(['Census Tract', 'Borough', 'Year', 'FIPS County Code', 'DCP Borough Code',\
                     'geometry'], axis=1)

In [ ]:
merged.to_csv('../data/bbl_ct_noise.csv')

In [ ]:
#ct_noise_pop.to_file('../data/ct_noise_pop/ct_noise_pop.shp',driver='ESRI Shapefile')